<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# NLP Using the Twitter API: Guided Lab

_Authors: Dave Yerrington (SF)_

---


<img src="https://snag.gy/RNAEgP.jpg" width="600">

### Can we correctly identify which of these two old men tweeted what?

> *Note: this lab is intended to be a guided lab until the independent practice questions.*


## Goals
---

We are going to attempt to classify whether a tweet comes from Trump or Sanders.  This lab involves multiple steps:
- Create a developer account on Twitter
- Create a method to pull a list of tweets from the Twitter API
- Perform proper preprocessing on our text
- Engineer sentiment feature in our dataset using TextBlob
- Explore supervised classification techniques


## Twitter API Developer Registration
---

If you haven't registered a Twitter account yet, this is a requirement in order to have a "developer" account.

[Twitter Rest API](https://dev.twitter.com/rest/public)



## Create an "App"

---

![](https://snag.gy/HPBQbJ.jpg)

Go to Twitter and register an "app" [apps.twitter.com](https://apps.twitter.com/).

> **Note**: For the required website field you can put a placeholder.

After you set up our app, you will only need to reference the cooresponding keys Twitter generates for our app.  These are the keys that we will use with our application to communicate with the Twitter API.

## Install Python Twitter API library

---

Someone was nice enough to build a Python libary for us. It makes pulling tweets simple: we only need to plug in our keys and start collecting data. The library we will be using is provided by [Python Twitter Tools](http://mike.verdone.ca/twitter/).

To install it, just run the next frame (there is no conda package).

In [1]:
!pip install twitter python-twitter

You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Some Boring Twitter Rules
---

**Twitter notifies you they will rate limit your requests:**

>When using application-only authentication, rate limits are determined globally for the entire application. If a method allows for 15 requests per rate limit window, then it allows you to make 15 requests per window — on behalf of your application. This limit is considered completely separately from per-user limits. https://dev.twitter.com/rest/public/rate-limiting

Here's a quick overview of what Twitter says are "the rules":

![](https://snag.gy/yJ6vIH.jpg)


## About those Keys: OAuth Review
---

![](https://g.twimg.com/dev/documentation/image/appauth_0.png)

## What's going on here?  Take a minute..

## Our Application Keys
---

Take note of your application keys you will use to connect to Twitter and mine tweets from the official Bernie Sanders and Donald Trump twitter accounts:

![](https://snag.gy/H1djQK.jpg)

## `TweetMiner` class structure

---

The following code will get you up and running, providing connectivity to twitter. The class has the ability to make requests and can eventually transform the JSON responses into DataFrames.

This is a great example of using object-oriented Python to organize our code!

> **Note:** "request_limit" is used in this class to limit the number of tweets that are pulled per instance request.  Setting it to something lower until you've worked the bugs out of your request, and captured the data you want, is essential to avoiding the rate limit blocks.

### Twitter API key setup

Fill the information below in with the keys for your account.

- **consumer_key** - Find this in your app page under the "Keys and Access Tokens"
- **consumer_secret** - Right under **consumer_key** in the "Keys and Access Tokens" tab
- **access_token_key** - You will need to click the button to generate tokens to get this
- **access_token_secret** - Also available after you generate tokens


In [2]:
import twitter, re, datetime, pandas as pd

twitter_keys = {
    'consumer_key':        'lA6UQplM5sxIAIr83ueUl9sgE',
    'consumer_secret':     'f94t4BD6Vj7aCVkX1qAIVwsP4x69J2vXvm61lTIuwb9GfmdsuP',
    'access_token_key':    '1203210464-dYE1FvoUx1GjVcoyok3U1brWDpELBJEcSNGC1OC',
    'access_token_secret': 'kuVli1j010RPdYTboz9iIyp8QxX6PqRApdi49baLqBgzo'
}

api = twitter.Api(
    consumer_key         =   twitter_keys['consumer_key'],
    consumer_secret      =   twitter_keys['consumer_secret'],
    access_token_key     =   twitter_keys['access_token_key'],
    access_token_secret  =   twitter_keys['access_token_secret']
)


In [3]:
class TweetMiner(object):

    result_limit    =   20    
    api             =   False
    data            =   []
    
    twitter_keys = {
        'consumer_key':        'KmN03M1X1pImZ43sqdIu4yfnE',
        'consumer_secret':     'ePlIrIX5VXbZnO7DBu1RbFlw5lOai9dQr9n5TZb6vxnIdrr5Fz',
        'access_token_key':    '185036086-Q7K5IjuSoQZJwSIqD0wyHf6t62iPKatmfaPkriAM',
        'access_token_secret': 'cYpQz3xWHQbLplOj8iSeiNSOmMcsTOXmWcKMrJ9buLj5d'
    }
    
    def __init__(self, keys_dict, api, result_limit = 20):
        
        self.api = api
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="dyerrington", mine_rewteets=False, max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, max_id=last_tweet_id - 1)        
            else:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit)
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'handle':          item.user.name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                }
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data

## Instantiate the class
---

Make sure you pass the keys dictionary and the api as arguments.

**Check:** call the object's `mine_user_tweets()` method, providing a user to pull the tweets of.

In [4]:
miner = TweetMiner(twitter_keys, api, result_limit=2)

In [5]:
sanders = miner.mine_user_tweets(user="berniesanders", max_pages=5)
donald = miner.mine_user_tweets(user="realDonaldTrump", max_pages=5)

In [6]:
print(sanders[0])

{'tweet_id': 1090427254981705729, 'handle': 'Bernie Sanders', 'retweet_count': 447, 'text': 'We must recommit ourselves to bringing all people together in the fight for a just and humane world. When we stand together, we can win.', 'mined_at': datetime.datetime(2019, 1, 29, 20, 59, 13, 380644), 'created_at': 'Wed Jan 30 01:51:31 +0000 2019'}


In [7]:
print(donald[0])

{'tweet_id': 1090244651578204160, 'handle': 'Donald J. Trump', 'retweet_count': 19280, 'text': 'A low level staffer that I hardly knew named Cliff Sims wrote yet another boring book based on made up stories and… https://t.co/SGN2UN9Ohw', 'mined_at': datetime.datetime(2019, 1, 29, 20, 59, 16, 970350), 'created_at': 'Tue Jan 29 13:45:55 +0000 2019'}


### Convert the tweet ouputs to a pandas DataFrame

> *Hint: this is as easy as passing it to the DataFrame constructor!*

In [8]:
pd.DataFrame(sanders).head()

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Wed Jan 30 01:51:31 +0000 2019,Bernie Sanders,2019-01-29 20:59:13.380644,447,We must recommit ourselves to bringing all peo...,1090427254981705729
1,Tue Jan 29 23:21:03 +0000 2019,Bernie Sanders,2019-01-29 20:59:13.380665,1386,Big Pharma is one of the wealthiest and most p...,1090389390579372032
2,Tue Jan 29 21:53:06 +0000 2019,Bernie Sanders,2019-01-29 20:59:15.258176,1957,"One of the major, untold stories of our time i...",1090367255710494725
3,Sat Jan 26 23:56:35 +0000 2019,Bernie Sanders,2019-01-29 20:59:15.258184,2639,Real and fundamental change never comes from t...,1089311166554349568
4,Sat Jan 26 22:24:54 +0000 2019,Bernie Sanders,2019-01-29 20:59:15.622173,1959,"In the twenty-first century, a public educatio...",1089288093495115776


##  Create the training data

---

Let's get our "mined" data from the Twitter API.  

1. Mine Trump tweets
- Create a tweet DataFrame
- Mine Sanders tweets
- Append the results to our DataFrame

In [9]:
# we only need to "instantiate" once.  Then we can call mine_user_tweets as much as we want.
miner = TweetMiner(twitter_keys, api, result_limit=400)
trump_tweets = miner.mine_user_tweets("realDonaldTrump")

In [10]:
trump_df = pd.DataFrame(trump_tweets)
print(trump_df.shape)

(1000, 6)


In [11]:
bernie_tweets = miner.mine_user_tweets('berniesanders')

In [12]:
bernie_df = pd.DataFrame(bernie_tweets) 
print(bernie_df.shape)

(1000, 6)


In [13]:
tweets = pd.concat([trump_df, bernie_df], axis=0)
tweets.shape

(2000, 6)

## Any interesting ngrams going on with Trump?
---

Set up a vectorizer from sklearn and fit the text of Trump's tweets with an ngram range from 2 to 4. Figure out what the most common ngrams are.

> **Note:** It's up to you whether you want to remove stopwords or not. How does keeping or removing stopwords affect the results?

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

# We can use the TfidfVectorizer to find ngrams for us
vect = TfidfVectorizer(ngram_range=(2,4))

# Pulls all of trumps tweet text's into one giant string
summaries = "".join(trump_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[('https co', 751),
 ('of the', 112),
 ('in the', 68),
 ('will be', 58),
 ('for the', 45),
 ('at the', 45),
 ('rt realdonaldtrump', 40),
 ('and the', 38),
 ('on the', 35),
 ('fake news', 33),
 ('our country', 33),
 ('border security', 33),
 ('thank you', 31),
 ('the democrats', 31),
 ('we are', 29),
 ('united states', 28),
 ('it is', 28),
 ('the wall', 28),
 ('to be', 28),
 ('to the', 27)]

### Look at the ngrams for Bernie Sanders

In [15]:
# We can use the TfidfVectorizer to find ngrams for us
vect = TfidfVectorizer(ngram_range=(2,4))

# Pulls all of trumps tweet text's into one giant string
summaries = "".join(bernie_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[('https co', 789),
 ('of the', 67),
 ('in the', 66),
 ('we must', 56),
 ('to the', 56),
 ('to be', 52),
 ('we can', 42),
 ('this country', 41),
 ('we have', 38),
 ('for the', 37),
 ('on the', 37),
 ('for all', 36),
 ('it is', 35),
 ('in this', 34),
 ('health care', 34),
 ('is not', 33),
 ('thank you', 30),
 ('bernie sanders', 30),
 ('is the', 29),
 ('and the', 27)]

## Processing the tweets and building a model

---

To do classfication we will need to convert the tweets into a set of features.

**You will need to:**
- Vectorize input text data.
- Intialize a model (try Logistic regression).
- Train / Predict / cross-validate.
- Evaluate the performance of the model.

> **Bonus:** you may have noticed that there are website links in the tweets. What additional preprocessing steps can you do before building the model?


In [16]:
!pip install textacy

    100% |████████████████████████████████| 61kB 997kB/s 
    100% |████████████████████████████████| 194kB 4.0MB/s 
  Running setup.py bdist_wheel for ujson ... done
  Stored in directory: /Users/gGandenberger/Library/Caches/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
Successfully built ujson
madeye 0.1.0 requires keras, which is not installed.
madeye 0.1.0 requires mlflow, which is not installed.
madeye 0.1.0 requires tensorflow, which is not installed.
swapi 0.1.3 has requirement requests==2.5.0, but you'll have requests 2.21.0 which is incompatible.
swapi 0.1.3 has requirement six==1.8.0, but you'll have six 1.12.0 which is incompatible.
swapi 0.1.3 has requirement ujson==1.33, but you'll have ujson 1.35 which is incompatible.
  Found existing installation: requests 2.5.0
    Uninstalling requests-2.5.0:
      Successfully uninstalled requests-2.5.0
  Found existing installation: ujson 1.33
    Uninstalling ujson-1.33:
      Successfully uninstalled ujson

In [17]:
# BONUS
# Using the textacy package to do some more comprehensive preprocessing
# http://textacy.readthedocs.io/en/latest/
import numpy as np
from textacy.preprocess import preprocess_text

tweet_text = tweets['text'].values
clean_text = [preprocess_text(x, 
                              fix_unicode=True, 
                              lowercase=True, 
                              transliterate=False,
                              no_urls=True, 
                              no_emails=True, 
                              no_phone_numbers=True, 
                              no_currency_symbols=True,
                              no_punct=True, 
                              no_accents=True)
              for x in tweet_text]

In [18]:
print(tweet_text[0:3])

['A low level staffer that I hardly knew named Cliff Sims wrote yet another boring book based on made up stories and… https://t.co/SGN2UN9Ohw'
 'How does Da Nang Dick (Blumenthal) serve on the Senate Judiciary Committee when he defrauded the American people ab… https://t.co/JBKLZ66qx6'
 'In the beautiful Midwest, windchill temperatures are reaching minus 60 degrees, the coldest ever recorded. In comin… https://t.co/FnaCaBA5Nz']


In [19]:
print(clean_text[0:3])

['a low level staffer that i hardly knew named cliff sims wrote yet another boring book based on made up stories and url', 'how does da nang dick blumenthal serve on the senate judiciary committee when he defrauded the american people ab url', 'in the beautiful midwest windchill temperatures are reaching minus 60 degrees the coldest ever recorded in comin url']


In [20]:
# target is the handle.
# make trump 1 and sanders 0
y = tweets['handle'].map(lambda x: 1 if x == 'Donald J. Trump' else 0).values

print(np.mean(y))

0.5


In [21]:
from sklearn.linear_model import LogisticRegression

# Preprocess our text data to Tfidf
tfv = TfidfVectorizer(ngram_range=(1,4), max_features=2000)
X = tfv.fit_transform(clean_text).todense()

print(X.shape)

(2000, 2000)


In [22]:
# cross-validate the accuracy:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(LogisticRegression(), X, y, cv=10)

print(accuracies)
print(np.mean(accuracies))

# Setup logistic regression (or try another classification method here)
estimator = LogisticRegression()
estimator.fit(X, y)


[0.875 0.895 0.925 0.925 0.875 0.905 0.865 0.9   0.885 0.915]
0.8965


/Users/gGandenberger/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/gGandenberger/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/gGandenberger/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/gGandenberger/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/gGandenberger/anaconda3/envs/py37/lib/python3.7/site-packages/skl

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [23]:
# Very good accuracy considering the baseline is 50%

## Check the predicted probability for a random Sanders and Trump tweet
---

Below are provided a couple of tweets from both Sanders and Trump. I'm sure you can figure out on your own which one is which.

Estimate the predicted probability of being trump for the two tweets.

In [24]:
# Prep our source as TfIdf vectors
source_test = [
    "Demanding that the wealthy and the powerful start paying their fair share of taxes that's exactly what the American people want.",
    "Crooked Hillary is spending tremendous amounts of Wall Street money on false ads against me. She is a very dishonest person!"
]

############
# NOTE:  Do not re-initialize the tfidf vectorizor or the feature space willbe overwritten and
# hence your transform will not match the number of features you trained your model on.
#
# This is why you only need to "transform" since you already "fit" previously
#
####

Xtest = tfv.transform(source_test)

# Predict using previously trained logist regression `estimator`
estimator.predict_proba(Xtest)

array([[0.64098369, 0.35901631],
       [0.29860157, 0.70139843]])

In [25]:
# The 1st column is probability of being Bernie, and 2nd Trump. The classifier is getting it right.

## Independent practice questions

---

### 1. Pull tweets for some new users.

Experiment with using more data.  The API will not like it if you blow through their limits - be careful.  Try to grab only what you need one time, then work on the copy of the objects that are returned.  

> Read the documentation about rate limits and see if you can get enough without hitting the rate limit.  Are there any options available in the API to avoid such a problem?

**Pull tweets for more than two different users of your choice.**

In [26]:
# We deviate from trump / sanders using student tweets here to illustrate the NLP pipeine with twitter data

twitter_handles = ["dril", "LaziestCanine",'ch000ch']
tweets = {}

for twitter_handle in twitter_handles:
    print("Mining tweets for: {}".format(twitter_handle))
    miner = TweetMiner(twitter_keys, api, result_limit=500)
    tweets[twitter_handle] = miner.mine_user_tweets(user=twitter_handle, max_pages=10)


Mining tweets for: dril
Mining tweets for: LaziestCanine
Mining tweets for: ch000ch


In [27]:
multi = pd.DataFrame(tweets['dril'])
multi = multi.append(pd.DataFrame(tweets['LaziestCanine']))
multi = multi.append(pd.DataFrame(tweets['ch000ch']))

print(multi.shape)

(5984, 6)


In [28]:
multi.handle.value_counts()

Lazy dog    1997
chuuch      1996
wint        1991
Name: handle, dtype: int64

### 2. Build a multi-class classification model to distinguish between the users.

Try a new type of model than we used before.

In [29]:
tweet_text = multi['text'].values
clean_text = [preprocess_text(x, fix_unicode=True, lowercase=True, transliterate=False,
                              no_urls=True, no_emails=True, no_phone_numbers=True, no_currency_symbols=True,
                              no_punct=True, no_accents=True)
              for x in tweet_text]

y = multi['handle'].map(lambda x: 0 if x == 'wint' else 1 if x == 'Lazy dog' else 2).values

In [30]:
tfv = TfidfVectorizer(ngram_range=(1,3), max_features=2500)
X = tfv.fit_transform(clean_text)

In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

rf = RandomForestClassifier(n_estimators=250, verbose=1)
knn = KNeighborsClassifier(n_neighbors=7)

rf.fit(X_train, y_train)
knn.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    4.0s finished


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=7, p=2,
           weights='uniform')

In [33]:
# Random forest score:
print('RF: {}'.format(rf.score(X_test, y_test)))
print('KNN: {}'.format(knn.score(X_test, y_test)))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    0.3s finished


RF: 0.7149220489977728
KNN: 0.40924276169265034


In [34]:
# Baseline score:
multi.handle.value_counts() / multi.shape[0]

Lazy dog    0.333723
chuuch      0.333556
wint        0.332721
Name: handle, dtype: float64

In [35]:
rf_yhat = knn.predict(X_test)

### 3. Make a confusion matrix and classification report.

In [36]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, rf_yhat))

              precision    recall  f1-score   support

           0       0.49      0.23      0.32       630
           1       0.36      0.74      0.49       593
           2       0.52      0.25      0.34       573

   micro avg       0.41      0.41      0.41      1796
   macro avg       0.46      0.41      0.38      1796
weighted avg       0.46      0.41      0.38      1796



In [37]:
# Confusion Matrix
print(confusion_matrix(y_test, rf_yhat))

[[148 410  72]
 [ 89 441  63]
 [ 67 360 146]]


### 4. What is the most and least "distinctive" tweets for each user?

To find this, identify the tweet that has the highest (correct) predicted probability of being that user's tweet for each user.

In [38]:
rf.fit(X, y)

pp = rf.predict_proba(X)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    6.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    0.8s finished


In [39]:
pp[0:5]

array([[0.67279663, 0.15868054, 0.16852283],
       [0.4709947 , 0.46231926, 0.06668605],
       [0.964     , 0.012     , 0.024     ],
       [0.936     , 0.048     , 0.016     ],
       [0.4709947 , 0.46231926, 0.06668605]])

In [40]:
pp = pd.DataFrame(pp, columns=['dril_pp', 'laziestcanine_pp', 'ch000ch_pp'])

In [41]:
print(multi.shape)
print(pp.shape)

(5984, 6)
(5984, 3)


In [42]:
tweets_pp = pd.concat([multi.reset_index(), pp.reset_index()], axis=1)
tweets_pp.head(2)

,index,created_at,handle,mined_at,retweet_count,text,tweet_id,index,dril_pp,laziestcanine_pp,ch000ch_pp
0,0,Wed Jan 30 00:16:24 +0000 2019,wint,2019-01-29 20:59:36.536890,4,@bicyclememes No thats too far .,1090403317333676033,0,0.672797,0.158681,0.168523
1,1,Wed Jan 30 00:06:37 +0000 2019,wint,2019-01-29 20:59:36.536899,775,https://t.co/wYLI2vf0pb,1090400854736138240,1,0.470995,0.462319,0.066686


In [43]:
print('Most dril: {}'.format(tweets_pp[tweets_pp.handle == 'wint'].sort_values('dril_pp', ascending=False).text.values[0]))
print('Least dril: {}'.format(tweets_pp[tweets_pp.handle == 'wint'].sort_values('dril_pp', ascending=True).text.values[0]))

Most dril: RT @fate_fox: あぁMeat Boll　おいしそう
Least dril: Kurnis


In [44]:
print('Most LaziestCanine: {}'.format(tweets_pp[tweets_pp.handle == 'Lazy dog'].sort_values('laziestcanine_pp', ascending=False).text.values[0]))
print('Least LaziestCanine: {}'.format(tweets_pp[tweets_pp.handle == 'Lazy dog'].sort_values('laziestcanine_pp', ascending=True).text.values[0]))

Most LaziestCanine: @AidanColt thamk uu!!
Least LaziestCanine: @J_Millie42 lmao


In [45]:
print('Most chuuch: {}'.format(tweets_pp[tweets_pp.handle == 'chuuch'].sort_values('ch000ch_pp', ascending=False).text.values[0]))
print('Least chuuch: {}'.format(tweets_pp[tweets_pp.handle == 'chuuch'].sort_values('ch000ch_pp', ascending=True).text.values[0]))

Most chuuch: @sixftbelle lol
Least chuuch: @MischaShriver https://t.co/KoKs8j1JNs
